## 매수추천 지표  
- PBR(주가순자산비율) : 동종업계 평균보다 높을 시 과열이 진정되면 하락할 가능성 다분.
- PER(주가수익비율) : 작으면 기업이 내는 순이익에 비해 저평가, 크면 주가가 고평가되어있으므로 과열상태를 의미

In [1]:
import urllib.parse
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import time
import datetime
import math
import FinanceDataReader as web
from datetime import date, timedelta
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import numpy as np

In [2]:
## 입력받은 데이터의 종목 코드를 확인
MARKET_CODE_DICT = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt',
    'konex': 'konexMkt'
}
DOWNLOAD_URL = 'https://kind.krx.co.kr/corpgeneral/corpList.do'
MARKET_CODE_DICT

{'kospi': 'stockMkt', 'kosdaq': 'kosdaqMkt', 'konex': 'konexMkt'}

In [3]:
##종목 코드에 맞춰서 zfill=자릿수만큼 앞에 0으로 채워주는 함수

def zeroFill(columnValue):
    columnValue = str(columnValue)
    outValue = columnValue.zfill(6)
    return outValue

In [4]:
## 상장폐지가 아닌 상장사들만 조회하려는 코드이다!
## input되는 key값에 맞는 value 값 전체가 return 됨
# 코스피 종목을 불러올 함수이다.

def get_stock_list(market=None, delisted=False):
    params = {'method': 'download'}

    if market.lower() in MARKET_CODE_DICT:
        ## marketType 키 추가
        params['marketType'] = MARKET_CODE_DICT[market]
        print(market.lower()+" market key is exist")

    else:
        print("invalid market")
        
    # make url  key=value & key = value

    params_string = urllib.parse.urlencode(params)
    request_url = DOWNLOAD_URL+"?"+params_string
    df = pd.read_html(request_url)[0]
    df["종목코드"] = df.종목코드.apply(zeroFill) 
    # df["종목코드"] = df.종목코드.map('{:06d}'.format) # 동일 결과
    return df

In [6]:
inputName = input("원하는 종목명 입력")

원하는 종목명 입력 카카오


In [7]:
stocks = get_stock_list('kospi')

# 종목명으로 코드검색
myStockNumber = stocks.loc[stocks['회사명'] == inputName]["종목코드"].iloc[0]
myStockNumber

kospi market key is exist


'035720'

In [8]:
## 유저가 입력한 주식종목의 코드 가져오기
def get_stock_number(stock_Name):
    stocks = stock_list('kospi')
    stocksRow = stocks.loc[stocks.회사명==stock_Name]
    code = stocksRow.iloc[0]['종목코드']
    return code

In [9]:
# 플랫폼 별 폰트설정
print(platform.system())

if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else:
    pass

Windows


In [10]:
# 네이버 금융에 들어가버리기
from bs4 import BeautifulSoup 
import requests 
import pandas as pd

url = 'https://finance.naver.com/item/main.nhn?code=' + myStockNumber
table_df_list = pd.read_html(url, encoding='euc-kr')    # 한글이 깨짐. utf-8도 깨짐. 그래서 'euc-kr'로 설정함  

earningInfo = table_df_list[3]
earningInfo

주요재무정보  최근 연간 실적                                 최근 분기 실적            \
       주요재무정보   2018.12   2019.12   2020.12 2021.12(E)   2020.03   2020.06   
       주요재무정보    IFRS연결    IFRS연결    IFRS연결     IFRS연결    IFRS연결    IFRS연결   
0         매출액  24170.00  30701.00  41568.00   56775.00   8684.00   9529.00   
1        영업이익    729.00   2068.00   4559.00    7845.00    882.00    978.00   
2       당기순이익    159.00  -3419.00   1734.00    8630.00    799.00   1452.00   
3       영업이익률      3.02      6.73     10.97      13.82     10.16     10.26   
4        순이익률      0.66    -11.14      4.17      15.20      9.20     15.24   
5   ROE(지배주주)      1.04     -5.81      2.70      11.77     -4.73     -2.67   
6        부채비율     41.45     52.21     60.94        NaN     57.01     49.86   
7        당좌비율    146.51    127.98    162.77        NaN    134.03    144.53   
8         유보율  12219.62  12027.79  13881.01        NaN  12587.68  13665.91   
9      EPS(원)    123.00   -717.00    355.00    1770.00    179.00    320.00   
10     PER(배)    168.01    -42.82    219.25      81.64    -52.50   -152.84   
11     BPS(원)  12979.00  12746.00  14647.00   16327.00  13108.00  14073.00   
12     PBR(배)      1.59      2.41      5.32       8.85      2.37      3.80   
13   주당배당금(원)     25.00     25.00     30.00     106.00       NaN       NaN   
14   시가배당률(%)      0.12      0.08      0.04        NaN       NaN       NaN   
15    배당성향(%)     20.99     -3.46      8.29        NaN       NaN       NaN   

                                             
     2020.09   2020.12   2021.03 2021.06(E)  
      IFRS연결    IFRS연결    IFRS연결     IFRS연결  
0   11004.00  12351.00  12580.00   13461.00  
1    1202.00   1497.00   1575.00    1789.00  
2    1437.00  -1955.00   2399.00    1985.00  
3      10.92     12.12     12.52      13.29  
4      13.06    -15.83     19.07      14.75  
5      -1.33      2.70      5.00        NaN  
6      50.37     60.94     71.56        NaN  
7     148.86    162.77    163.55        NaN  
8   14373.52  13881.01  14508.10        NaN  
9     290.00   -428.00    506.00     445.00  
10   -404.46    219.25    144.96     324.73  
11  14910.00  14647.00  15428.00        NaN  
12      4.89      5.32      6.46        NaN  
13       NaN       NaN       NaN        NaN  
14       NaN       NaN       NaN        NaN  
15       NaN       NaN       NaN        NaN

In [11]:
# NaN값 0으로 채움
earningInfo = earningInfo.fillna(0)
earningInfo

주요재무정보  최근 연간 실적                                 최근 분기 실적            \
       주요재무정보   2018.12   2019.12   2020.12 2021.12(E)   2020.03   2020.06   
       주요재무정보    IFRS연결    IFRS연결    IFRS연결     IFRS연결    IFRS연결    IFRS연결   
0         매출액  24170.00  30701.00  41568.00   56775.00   8684.00   9529.00   
1        영업이익    729.00   2068.00   4559.00    7845.00    882.00    978.00   
2       당기순이익    159.00  -3419.00   1734.00    8630.00    799.00   1452.00   
3       영업이익률      3.02      6.73     10.97      13.82     10.16     10.26   
4        순이익률      0.66    -11.14      4.17      15.20      9.20     15.24   
5   ROE(지배주주)      1.04     -5.81      2.70      11.77     -4.73     -2.67   
6        부채비율     41.45     52.21     60.94       0.00     57.01     49.86   
7        당좌비율    146.51    127.98    162.77       0.00    134.03    144.53   
8         유보율  12219.62  12027.79  13881.01       0.00  12587.68  13665.91   
9      EPS(원)    123.00   -717.00    355.00    1770.00    179.00    320.00   
10     PER(배)    168.01    -42.82    219.25      81.64    -52.50   -152.84   
11     BPS(원)  12979.00  12746.00  14647.00   16327.00  13108.00  14073.00   
12     PBR(배)      1.59      2.41      5.32       8.85      2.37      3.80   
13   주당배당금(원)     25.00     25.00     30.00     106.00      0.00      0.00   
14   시가배당률(%)      0.12      0.08      0.04       0.00      0.00      0.00   
15    배당성향(%)     20.99     -3.46      8.29       0.00      0.00      0.00   

                                             
     2020.09   2020.12   2021.03 2021.06(E)  
      IFRS연결    IFRS연결    IFRS연결     IFRS연결  
0   11004.00  12351.00  12580.00   13461.00  
1    1202.00   1497.00   1575.00    1789.00  
2    1437.00  -1955.00   2399.00    1985.00  
3      10.92     12.12     12.52      13.29  
4      13.06    -15.83     19.07      14.75  
5      -1.33      2.70      5.00       0.00  
6      50.37     60.94     71.56       0.00  
7     148.86    162.77    163.55       0.00  
8   14373.52  13881.01  14508.10       0.00  
9     290.00   -428.00    506.00     445.00  
10   -404.46    219.25    144.96     324.73  
11  14910.00  14647.00  15428.00       0.00  
12      4.89      5.32      6.46       0.00  
13      0.00      0.00      0.00       0.00  
14      0.00      0.00      0.00       0.00  
15      0.00      0.00      0.00       0.00

In [33]:
# 타겟주식 제일 최근의 PBR값 가져옴
targetRecntPBR = earningInfo.iloc[12,9]
targetRecntPBR

6.46

In [31]:
# 동종업계 정보표시 df
sameBusinessInfo = table_df_list[4]
sameBusinessInfo

,종목명,카카오*035720,NAVER*035420,아프리카TV*067160,키다리스튜디오*020120,자이언트스텝*289220
0,현재가,143000,391000,103500,15000,47950
1,전일대비,"하향 1,500","상향 4,000",하향 500,상향 200,상향 250
2,등락률,하향 -1.04%,상향 +1.03%,하향 -0.48%,상향 +1.35%,상향 +0.52%
3,시가총액(억),634819,642269,11897,4971,4562
4,외국인비율(%),33.42,56.80,50.06,1.08,.53
5,매출액(억),12580,14991,609,199,70
6,영업이익(억),1575,2888,196,23,-5
7,조정영업이익(억),1575,2888,196,23,-5
8,영업이익증가율(%),5.27,-10.79,10.67,285.57,-319.72
9,당기순이익(억),2399,153145,154,19,-4


In [35]:
# 타겟주식을 제외한 동종업계 종목들 PBR 평균구하기
samebusinessPBR = sameBusinessInfo.iloc[13][2:].astype(float).mean()
samebusinessPBR

6.890000000000001

In [36]:
goodstockCnt = 0;
if targetRecntPBR <= samebusinessPBR:
    goodstockCnt = goodstockCnt + 1;

In [37]:
# 타겟 주식 PER (2020.03 ~ 2021.03)
targetStockAvgPER = float(earningInfo.values[10][5:-1].mean())

# 동일업종 PER 나와있는 부분 df로 저장
sameBusniessPER = table_df_list[9]

# 동일업종 PER 값 글자나누고 float으로 바꿈
targetBusniessPER = sameBusniessPER[1].values[0]
targetBusniessPER = float(targetBusniessPER.split("배")[0])
targetBusniessPER


if targetStockAvgPER <= targetBusniessPER:
    goodstockCnt = goodstockCnt + 1;

In [38]:
if goodstockCnt == 0:
    print("돈을 잃고 싶다면야 말리진 않겠습니다...")
elif goodstockCnt == 1:
    print("충분히 걸만한 투자입니다")
elif goodstockCnt == 2:
    print("투자하십시오! 다만 책임은 지지 않습니다")

투자하십시오! 다만 책임은 지지 않습니다


In [ ]:
def comparePER(stockCode):


    # stock_code = '5490'
    stock_code = str(stockCode).zfill(6)

    url = 'https://finance.naver.com/item/main.nhn?code=' + stock_code
    table_df_list = pd.read_html(url, encoding='euc-kr')
    earningInfo = table_df_list[3]

    targetStockAvgPER = float(earningInfo.values[10][5:-1].mean())
    sameBusniessPER = table_df_list[9]
    targetBusniessPER = sameBusniessPER[1].values[0]
    targetBusniessPER = float(targetBusniessPER.split("배")[0])

    if targetStockAvgPER <= targetBusniessPER:
        return 1
    else:
        return 2